<a href="https://colab.research.google.com/github/anshikabajpai23/Energy-Utilization-of-Rooftops-In-Urban-Areas/blob/FilterPlants/Filtering-Plants/FilteredPlants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# @markdown Loading Dependencies
%%capture installation
!pip install pyrebase
!pip install patool
!pip install pgeocode

from datetime import datetime
import math
from bs4 import BeautifulSoup as bs
import requests
import pgeocode
import pyrebase
import json
import patoolib
from distutils.dir_util import copy_tree
import os
import cv2
import decimal
import numpy as np
import pandas as pd
from keras.preprocessing.image import save_img 
from PIL import Image


In [0]:

patoolib.extract_archive("/content/gdrive/My Drive/nerve.rar", outdir="/content")
src = '/content/nerve/nerve/model'
dst = '/content/model'
copy_tree(src, dst)
from model.u_net import get_unet_256


In [0]:
# @markdown Meta-Data
#add details according to your account on firebase
%%file configshare.json
{
  
}

Overwriting configshare.json


In [0]:
# @markdown Connecting with Firebase

class Firebase:
    def __init__(self,configFilePath):
        '''
            Please allow read and write privilege to all 
        '''
        config_data = open(configFilePath,"r")
        config = json.load(config_data)
        self.firebase = pyrebase.initialize_app(config)
        self.db =  self.firebase.database()
        self.storage = self.firebase.storage()
    def downloadLatestImage(self):
        self.img_from_fb = []
        images = [image.val() for image in self.db.child('uploadedFromPython').get().each()]
        print(images[len(images)-1])
        for i in range(4):
          img_name = str(images[len(images)-1-i])
          self.img_from_fb.append(img_name)
          self.downloadImageFromFireBase('uploadedFromPython/'+img_name,img_name)
    def downloadImageFromFireBase(self,path,saveName):
        '''
            folder fetches image from url
            and saves as downloads.jpeg
            path example: images/one.jpeg
        '''
        self.storage.child(path).download(saveName)
    def uploadImageFromFireBase(self,imagePath,imageName):
        '''
            imagePath is the path of image to be uploaded
            imageName is the name reflected on firebase in uploadedFromPython
        '''
        uploadPath = 'segmentedImages/'+imageName
        self.storage.child(uploadPath).put(imagePath)
        self.db.child('segmentedImages').push(imageName)
    def getLat(self):
        #lat =[lat.val() for lat in self.db.child('uploadedLat').get().each()]
        lat = self.db.child('uploadedLat').get()
        self.latitude = lat.val()
        return self.latitude
    def getLon(self):
        #lon =[lon.val() for lon in self.db.child('uploadedLon').get().each()]
        lon = self.db.child('uploadedLon').get()
        self.longitude = lon.val()
        return self.longitude
    def getArea(self):
        areaOfRooftop = self.db.child('uploadedArea').get()
        self.area = areaOfRooftop.val()
        return self.area
    def getPostalCode(self):
        postalCode = self.db.child('uploadedPostalCode').get()
        self.postal = postalCode.val()
        return self.postal
    def saveResultToDatabase(self,result):
        self.db.child('result').push(result)
obj  = Firebase("configshare.json")
obj.downloadLatestImage()

3.jpg


In [0]:
print(obj.getLat())
print(obj.getLon())
print(obj.getArea())
print(obj.getPostalCode())

26.8879745
80.9337338
10
226020


In [0]:
obj.img_from_fb

['3.jpg', '2.jpg', '1.jpg', '0.jpg']

In [0]:
# @markdown IMAGE SEGMENTATION
model = get_unet_256()
data_dir = '/content'
input_size = 256 
epochs = 5
batch_size = 16

orig_width = 580
orig_height = 420

threshold = 0.5
rles = []
model.load_weights('/content/gdrive/My Drive/minor/segmentation_weights2.h5')
x_batch=[]

#img_from_fb = ['2.png','3.png','5.png','6.png']

def run_length_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.transpose().flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

for i in obj.img_from_fb:
  img_path = '/content/' + i
  #img_path = '/content/' + i
  img = cv2.imread(img_path)
  img = cv2.resize(img, (input_size, input_size))
  x_batch.append(img)

x_batch = np.array(x_batch, np.float32) / 255
preds = model.predict(x_batch)
preds = np.squeeze(preds, axis=3)

for pred in preds:
  prob = cv2.resize(pred, (orig_width, orig_height))
  mask = prob > threshold
  rle = run_length_encode(mask)
  rles.append(rle)
#prob = cv2.resize(preds[0], (orig_width, orig_height))
#mask = prob > threshold
#rle = run_length_encode(mask)

def rle_decode(mask_rle, shape=(orig_width,orig_height)):
    #print('rle_decode(mask_rle = ', mask_rle)
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

mylist = img_path.split('/')
mylist.pop()
file_path = '/'.join(mylist)
masks_path = []
for k in range(len(rles)):
  imgg= rle_decode(rles[k])
  output = 'mask_' + obj.img_from_fb[k]
  #output = 'mask_' + img_from_fb[k]
  out_path = os.path.join(file_path, output)
  masks_path.append(out_path)
  save_img(out_path, imgg[..., np.newaxis])
  print('Image Saved Succesfully at {} location with image name : {}'.format(file_path,output))
  obj.uploadImageFromFireBase(out_path,output)


Image Saved Succesfully at /content location with image name : mask_3.jpg
Image Saved Succesfully at /content location with image name : mask_2.jpg
Image Saved Succesfully at /content location with image name : mask_1.jpg
Image Saved Succesfully at /content location with image name : mask_0.jpg


In [0]:
# @markdown Distance Calculator 
def calculateDist(lati1,long1,lati2,long2):
  R = 6373.00

  lat1 = math.radians(round(float(lati1),2))
  lon1 = math.radians(round(float(long1),2))
  lat2 = math.radians(lati2)
  lon2 = math.radians(long2)

  dlon = lon2 - lon1
  dlat = lat2 - lat1

  a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
  distance = R * c
#distance is in km
  #print(distance)
  return distance

In [0]:
# @markdown Calculations of Solar Potential, Temperature and Energy

#list_im = ['1.jpg', '2.jpg', '3.jpg','4.jpg']
imgs    = [ Image.open(i) for i in masks_path ]
# pick the image which is the smallest, and resize the others to match it (can be arbitrary image shape here)
min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
imgs_comb = np.hstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )

# save that beautiful picture
imgs_comb = Image.fromarray( imgs_comb)
imgs_comb.save( 'sky_horizontal.jpg' )    

# for a vertical stacking it is simple: use vstack
imgs_comb = np.vstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
imgs_comb = Image.fromarray( imgs_comb)
imgs_comb.save( 'sky_vertical.jpg' )
#calculating fraction
#img = cv2.imread(output, 0)
img = cv2.imread('sky_horizontal.jpg', 0)
c = 0
x, y = img.shape
for i in range(0,x):
  for j in range(0,y):
    if img[i][j] != 0:
      c = c+1
rounded = round(c/(x*y), 1)

#calculating state name
nomi = pgeocode.Nominatim('in')
var = nomi.query_postal_code(obj.getPostalCode())
#var = nomi.query_postal_code('412207')
state_name = var.state_name
stateName = state_name.lower()
print("State name: {}".format(stateName))
stateName= stateName.split(' ')[0]
#print(stateName)
#calculating current month 
currentMonth = datetime.now().month
print('Current month in numerical value is {}'.format(currentMonth))
Months = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']

df = pd.read_csv('/content/gdrive/My Drive/minor/Irradiance_Dataset2.csv')
stateName_index = []
for i in df.index:
  if df['State'][i].startswith(stateName):
    stateName_index.append(i)
#print(stateName_index)
df = df.take(stateName_index)
df2 = df[df['Month']==Months[currentMonth-1]]
distances = []

#calculation of irradiance values
for i in df2.index:
  distances.append(calculateDist(obj.getLat(),obj.getLon(),df2['Latitude'][i],df2['Longitude'][i]))

index = df2.index[distances.index(min(distances))]
closestDistrict = df2['District'][index]
irradianceVal = df2['Irradiance'][index]
temperature = df2['Temperature'][index]

#formula for calculating solar potential in kWh per square meter per day
solarPotential = irradianceVal * float(obj.getArea()) * rounded 

#formula for calculating Energy in Joules per square meter
energy = solarPotential * 3600 * 1000

#Since,Crops / plants absorb about 2% of solar energy falled on them
energyPlants = energy * 0.02

State name: maharashtra
maharashtra
Current month in numerical value is 5
[708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 

In [0]:
plantsData = pd.read_excel('/content/gdrive/My Drive/minor/plants dataset.xlsx')
plantsData.head(2)

,plants,soil,min temp,max temp,energy/ sq. m in Joules,weather,months,image
0,Tomato,Loam and Sandy Loam,18,24,409680,Warm / Summer,Mar - Jun,https://www.growjoy.com/store/pc/catalog/manal...
1,Spinach,Loam,10,22,417600,All,Sep - Jun,https://hips.hearstapps.com/hmg-prod.s3.amazon...


In [0]:
# @markdown Filter Plants that satisfies all necessary conditions
plantIndex = []
satisfiesTemp = set()
satisfiesEnergy = set()
plantsDistrict = df[df['District']==closestDistrict]
name=''
if rounded >= 0.3:
  for i in plantsData.index:
    if int(energyPlants) >= (plantsData['energy/ sq. m in Joules'][i] * float(obj.getArea())):
      satisfiesEnergy.add(i)
  for h in plantsDistrict.index:
    plants = []
    for ind in list(satisfiesEnergy):
      if plantsDistrict['Temperature'][h] >= plantsData['min temp'][ind] and plantsDistrict['Temperature'][h] <= plantsData['max temp'][ind]:
        plants.append(ind)
      #print(plants)
    best_plants[plantsDistrict['Month'][h]] = plants
  for key,value in best_plants.items():
    plants_list = "_".join(list(map(str,value)))
    name = name + '(' + key + ')' + plants_list
else:
  name = "NULL"

obj.saveResultToDatabase(name)

In [0]:
plantsDistrict

,Unnamed: 0,State,District,Latitude,Longitude,Month,Irradiance,Temperature
528,528,rajasthan,Jaipur,26.95,75.75,jan,4.22,16
529,529,rajasthan,Jaipur,26.95,75.75,feb,5.23,20
530,530,rajasthan,Jaipur,26.95,75.75,mar,6.26,25
531,531,rajasthan,Jaipur,26.95,75.75,apr,7.02,31
532,532,rajasthan,Jaipur,26.95,75.75,may,7.43,34
533,533,rajasthan,Jaipur,26.95,75.75,jun,6.61,34
534,534,rajasthan,Jaipur,26.95,75.75,jul,5.91,31
535,535,rajasthan,Jaipur,26.95,75.75,aug,5.37,29
536,536,rajasthan,Jaipur,26.95,75.75,sep,5.96,29
537,537,rajasthan,Jaipur,26.95,75.75,oct,5.55,28


In [0]:
name

'(jan)13_22_26_28_29(feb)13_14_23_24_25_27_28_29(mar)14_24_25_27_28(apr)24_27(may)24_27(jun)27(jul)24_27(aug)24_27(sep)24_27(oct)24_25_27(nov)13_14_23_24_25_27_28_29(dec)13_14_23_25_28_29'

In [0]:
for h in plantsDistrict.index:
  plants = []
  for ind in list(satisfiesEnergy):
    if plantsDistrict['Temperature'][h] >= plantsData['min temp'][ind] and plantsDistrict['Temperature'][h] <= plantsData['max temp'][ind]:
      plants.append(plantsData['plants'][ind])
      #print(plants)
  best_plants[plantsDistrict['Month'][h]] = plants
print(best_plants)

{'jan': ['Hibiscus', 'Bougainvillea', 'Allamanda', 'Plumerias', 'Rangoon Creeper'], 'feb': ['Hibiscus', 'Jasmine', 'Gardenias', 'Tuberose', 'Rose', 'Marigold', 'Plumerias', 'Rangoon Creeper'], 'mar': ['Jasmine', 'Tuberose', 'Rose', 'Marigold', 'Plumerias'], 'apr': ['Tuberose', 'Marigold'], 'may': ['Tuberose', 'Marigold'], 'jun': ['Marigold'], 'jul': ['Tuberose', 'Marigold'], 'aug': ['Tuberose', 'Marigold'], 'sep': ['Tuberose', 'Marigold'], 'oct': ['Tuberose', 'Rose', 'Marigold'], 'nov': ['Hibiscus', 'Jasmine', 'Gardenias', 'Tuberose', 'Rose', 'Marigold', 'Plumerias', 'Rangoon Creeper'], 'dec': ['Hibiscus', 'Jasmine', 'Gardenias', 'Rose', 'Plumerias', 'Rangoon Creeper']}


# **DO NOT RUN CODE AFTER THIS**
This code is for *collage*

In [0]:
# @markdown IMAGE SEGMENTATION


model = get_unet_256()
data_dir = '/content'
input_size = 256
epochs = 5
batch_size = 16

orig_width = 580
orig_height = 420

threshold = 0.5

model.load_weights('/content/gdrive/My Drive/minor/segmentation_weights2.h5')
x_batch=[]

def run_length_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.transpose().flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


img_path = '/content/' + obj.img_from_fb
img = cv2.imread(img_path)
img = cv2.resize(img, (input_size, input_size))
x_batch.append(img)
x_batch = np.array(x_batch, np.float32) / 255
preds = model.predict(x_batch)
preds = np.squeeze(preds, axis=3)
prob = cv2.resize(preds[0], (orig_width, orig_height))
mask = prob > threshold
rle = run_length_encode(mask)

def rle_decode(mask_rle, shape=(orig_width,orig_height)):
    #print('rle_decode(mask_rle = ', mask_rle)
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

imgg= rle_decode(rle)
mylist = img_path.split('/')
mylist.pop()
file_path = '/'.join(mylist)
output = 'mask' + obj.img_from_fb
out_path = os.path.join(file_path, output)
save_img(out_path, imgg[..., np.newaxis])
print('Image Saved Succesfully at {} location with image name : {}'.format(file_path,output))
obj.uploadImageFromFireBase(out_path,output)


Image Saved Succesfully at /content location with image name : maskIMG_20200518_133614.png
